In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright,TimeoutError as PlayWrightError

In [2]:
import time

In [3]:
url='https://stats.espncricinfo.com/ci/engine/stats/index.html'
#https://www.espncricinfo.com/records/trophy/indian-premier-league-117

In [4]:
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15'}

In [5]:
async def get_html_content(url,selector,retries=3,sleep=5):
    html=None
    for i in range(1,retries+1):
        time.sleep(sleep*i)
        try:
            async with async_playwright() as p:
                #Opens browser
                browser=await p.chromium.launch()
                #opens a new tab
                page= await browser.new_page()
                #will search for the  url
                await page.goto(url)
                print(await page.title())
                #gathers the html data
                html=await page.inner_html(selector)
                await browser.close()
              
        except PlayWrightError:
             print(f'issue with the given {url}')
             continue
        
        break
    return html
     
            
        
        
    
    

In [6]:
from urllib.parse import urljoin

In [7]:
async def get_html_content2(url, selector, retries=3, sleep=5):
    html_list = []  # To store the HTML of all matching elements
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                #opens browser
                browser = await p.chromium.launch()
                #opens new page
                page = await browser.new_page()
                #will search for provided url
                await page.goto(url)
                print(await page.title())

                # Use page.locator() to get all matching elements
                elements = await page.locator(selector).all()  # Fetch all matching elements
                
                # Loop through each element and retrieve the inner HTML
                for element in elements:
                    html = await element.inner_html()
                    html_list.append(html)
                
                await browser.close()

        except PlaywrightError:
            print(f'Issue with the given {url}')
            continue
        
        break
    return html_list  # Return a list of inner HTML for all matching elements



starting url: https://stats.espncricinfo.com/ci/engine/stats/index.html

In [8]:
#From the starting url grabs all the tabs
html_content=await get_html_content(url,'#subnav_tier1.subnav_tier1.subnav-item-wrap')
html_content= BeautifulSoup(html_content,'html.parser')
from urllib.parse import urljoin
#Filter out the ipl tab
hrefs= [each['href'] for each in html_content.find_all('a') if each and each.text.strip().lower()=="ipl"]
url2=urljoin("https://www.espncricinfo.com",hrefs[0])
#gives a list of html that matches the selector
html_list=await get_html_content2(url2,'ul.ds-flex.ds-flex-col')
print(len(html_list))
list_of_links=[]
#The below loop gives the links of various tabs
for index,each in enumerate(html_list):
    html_1=BeautifulSoup(each)
    #print(html_1)
    #print('The next one is')
    #print("  ")
    for ind in html_1.find_all('a'):
        list_of_links.append(ind['href'])
#filter the list

        
        
        
    
    







Statsguru | Searchable Cricket Statistics database | ESPNcricinfo.com
Indian Premier League Records - Cricket's Remarkable Feats
12


In [9]:
batting_filtered_lists=[]
bowling_filtered_lists=[]
for each in list_of_links:
    #filter only batting averages list
    if 'team' in each and 'averages' in each and 'batting' in each:
        #https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4343
        base='https://www.espncricinfo.com'
        each=urljoin(base,each)
        batting_filtered_lists.append(each)
    if 'team' in each and 'averages' in each and 'bowling' in each:
        base='https://www.espncricinfo.com'
        each=urljoin(base,each)
        bowling_filtered_lists.append(each)

In [10]:
batting_filtered_lists

['https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4343',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4347',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4344',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=5845',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=6904',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4342',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4788',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=4341',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?team=6903',
 'https://www.espncricinfo.com/records/trophy/averages-batting/indian-premier-league-117?te

In [11]:
bowling_filtered_lists

['https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4343',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4347',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4344',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=5845',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=6904',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4342',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4788',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4341',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=6903',
 'https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?te

In [35]:
async def getting_averages(list1):
    print(list1[0])
    if 'batting' in list1[0]:
      batting_list=[]
    else:
        bowling_list=[]
    for each_link in list1:
        #gets the table from the given link
        table_content=await get_html_content(each_link,'table')
        #To capture the team name
        table_heading=await get_html_content(each_link,'h1.ds-text-title-m.ds-font-bold.ds-my-4')
        table_heading=BeautifulSoup(table_heading,'html.parser')
        table_heading=str(table_heading).split('-')[1]
        table_content=BeautifulSoup(table_content,'html.parser')
        #to get the headers of the table
        table_head= table_content.find_all('thead')
        table_body= table_content.find_all('tbody')
        #stores all the headers in a list
        headers_list=[]
        for each_span in table_head[0].find_all('span'):
            headers_list.append(each_span.text.strip())
        #print(headers_list)
        #table_heading=str(table_heading).split('-')[1].strip()
        for i in table_body[0].find_all('tr'):
            #print(i)
            players_list=[]
            #print('going to loop')
            for each in i.find_all('span'):     
                #players_dict[headers_list[count]]=each.text.strip()
                players_list.append(each.text.strip())
                #print("The next one")
            #print(players_list)
            #As player name captures twice removing  the first one.
            players_list.pop(0)
            #print(players_list)
            players_dict={}
            for x,y in zip(headers_list,players_list):
                players_dict[x]=y
            players_dict['team_name']= table_heading
            if 'batting' in list[0]:
                batting_list.append(players_dict)
                #print(batting_list)
                
            else:
                bowling_list.append(players_dict)
                
                #print(bowling_list)
    if 'batting' in list[0]:
          return batting_list
    else:
          return bowling_list


In [28]:
batting_averages=await getting_averages(batting_filtered_lists)


Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Trophy averages batting | ESPNcricinfo
Indian Premier League Tr

In [36]:
bowling_averages=await getting_averages(bowling_filtered_lists)

https://www.espncricinfo.com/records/trophy/averages-bowling/indian-premier-league-117?team=4343
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | ESPNcricinfo
Indian Premier League Trophy averages bowling | E

In [30]:
ipl_batting_averages=pd.DataFrame(batting_averages)

In [31]:
ipl_batting_averages.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,SR,100,50,0,team_name
0,P Amarnath,2008-2008,6,-,-,-,-,0.00,0.00,-,-,-,Chennai Super Kings
1,Akash Singh,2023-2023,6,-,-,-,-,0.00,0.00,-,-,-,Chennai Super Kings
2,MM Ali,2021-2024,48,40,4,853,93,23.69,136.26,-,3,4,Chennai Super Kings
3,S Anirudha,2008-2013,19,12,5,133,64,19.00,123.14,-,1,2,Chennai Super Kings
4,KB Arun Karthik,2010-2010,1,1,1,3,3*,0.00,60.00,-,-,-,Chennai Super Kings


In [32]:
ipl_batting_averages.shape

(1386, 13)

In [37]:
ipl_bowling_averages=pd.DataFrame(bowling_averages)

In [38]:
ipl_bowling_averages.head()

,Player,Span,Mat,Inns,Balls,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,5,10,Ct,St,team_name
0,P Amarnath,2008-2008,6,6,132,-,236,7,2/29,33.71,10.72,18.8,-,-,2,-,Chennai Super Kings
1,Akash Singh,2023-2023,6,6,114,-,188,5,2/40,37.60,9.89,22.8,-,-,1,-,Chennai Super Kings
2,MM Ali,2021-2024,48,35,499,-,585,25,4/26,23.40,7.03,19.9,-,-,18,-,Chennai Super Kings
3,S Anirudha,2008-2013,19,-,0,-,-,-,-,0.00,-,-,-,-,11,-,Chennai Super Kings
4,KB Arun Karthik,2010-2010,1,-,0,-,-,-,-,0.00,-,-,-,-,-,-,Chennai Super Kings


In [39]:
ipl_bowling_averages.shape

(1386, 17)

In [42]:
ipl_bowling_averages.to_csv('ipl_bowling_averages.csv')

In [43]:
ipl_batting_averages.to_csv('ipl_batting_averages.csv')

If you want you can also keep other two blocks in the function